# **This notebook contains example on the usage of the class ``StructureFactor``**:
The data should be load as an object of type ``PointPattern``. This can be done using the calss ``PointPattern``. 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime  # must be added to dependencies
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ``HomogeneousPoissonPointProcess``:
The class ``HomogeneousPoissonPointProcess`` allows to sample a Poisson point process using ``generate_sample`` in a box window using ``BoxWindow`` of ``spatial_windows`` or a in a ball window using ``BallWindow``of ``spatial_windows``.

In [ ]:
from structure_factor.homogeneous_poisson_process import HomogeneousPoissonPointProcess
from structure_factor.spatial_windows import BoxWindow

L_poisson = 40 #length side of the cube containing the Poisson point process
bounds = np.array([[-L_poisson/2, -L_poisson/2], [L_poisson/2, L_poisson/2]]) #bounds of the box window containing the point process
window = BoxWindow(bounds) #box window containing the point process

intensity_poisson = 3 #intensity of the Poisson point process
process = HomogeneousPoissonPointProcess(intensity=intensity_poisson) 
points_1 = process.generate_sample(window=window) #Poisson point process in the cubic window

plt.plot(points_1[:,0], points_1[:,1], 'b,')
plt.show()

## ``PointPattern``:
The class ``StructureFactor``take an object of type ``PointPattern``.
The following is an exmaple of tansfering the sample of the Poisson point proces ``points_1`` to an objects of type ``PointPattern``.
The objects of type ``PointPattern`` have 3 attributes "points", "window" and "intensity". 

In [ ]:
from structure_factor.point_pattern import PointPattern
poisson_data_1 = PointPattern(points_1, window, intensity=intensity_poisson) #making the poisson point proces an object of type Point Pattern

Generating a Poisson point process in a disk using ``HomogeneousPoissonPointProcess``with ``BallWindow``.

In [ ]:
from structure_factor.spatial_windows import BallWindow
R_poisson = 70 #radius of the disk containing the Poisson point process
center_poisson = [0,0] #center of the disk contaniing the Poisson point process

window = BallWindow(center_poisson, R_poisson) #creating ball window
process = HomogeneousPoissonPointProcess() 
points_2 = process.generate_sample(window=window) #Poisson point process in the ball window

plt.plot(points_2[:,0], points_2[:,1], 'b,')
plt.show()

## ``PointPattern.restrict_to_cubic_window``:
The method ``compoute_scattering_intensity`` of the class ``StructureFactor`` works for ``PointPattern`` with **cubic** window. So we create the method ``restrict_to_window`` to restrict the PointPattern to a cubic window for the perpose of use in ``compoute_scattering_intensity``.

###``restrict_to_window``:

The following is an example of transforming the poisson point process ``points``to an object of type ``PointPattern`` restricted to a cubic window using the method ``restrict_to_window`` of the class ``PointPattern``.

In [ ]:
Poisson_data_2 = PointPattern(points_2) #to an object of type PointPattern

L_poisson_2 = int(R_poisson/(np.sqrt(2))) #length of the cubic window 
bounds = np.array([[-L_poisson_2/2, -L_poisson_2/2], [L_poisson_2/2, L_poisson_2/2]])
poisson_data_2 = Poisson_data_2.restrict_to_window(bounds) #restrict to a cubic window

plt.plot(poisson_data_2.points[:,0], poisson_data_2.points[:,1], 'b,')
plt.show()

# loading data from the saved file "test_data.dat":
Poisson point process ("poisson_data") , Ginibre point pross ("ginibre_data") , lattice $\mathbb{Z}^2$ ("z_2_data") and matching processus of Michael Andreas Klatt, Günter Last, D. Yogeshwaran that we will denoted by kly ("kly_data") defined in https://arxiv.org/abs/1810.00265

In [ ]:
import pickle
path_data = "../data"
with open(os.path.join(path_data, "test_data.dat"), "rb") as data:
    poisson_data, ginibre_data, kly_data, z_2_data = pickle.load(data, encoding="bytes")

The data in "test_data.dat": "poisson_data", "ginibre_data", "kly_data" and "z_2_data",  are objects of type ``PointPattern`` so they have 3 attributes "points", "window" and "intensity". 
For example ``poisson_data.points`` are the points of a Poisson point process  and ``poisson_data.window`` is the window containing the points, and ``poisson_data.intensity`` is the intensity of the point process.
``poisson_data.points``, ``kly_data.points`` and ``z_2_data.points`` lie in "BoxWindow", to see the bound of the window use for exmaple ``poisson_data.window.bounds``. while ``ginibre_data.points`` lie in a "BallWindow", to see the center and raduis use ``ginibre_data.window.center`` resp. ``ginibre_data.window.raduis``.

In [ ]:
print("poisson_data : point", poisson_data.points.shape , ", intensity=", poisson_data.intensity)
print("window of poisson_data :bounds ",  kly_data.window.bounds )
print("window of ginibre_data: center ",ginibre_data.window.center, ", raduis=", ginibre_data.window.radius)

#  ``StructureFactor``:

In [ ]:
from structure_factor.structure_factor_new import StructureFactor

In [ ]:
sf_poisson= StructureFactor(poisson_data)
sf_ginibre = StructureFactor(ginibre_data)
sf_kly = StructureFactor(kly_data)
sf_z_2 = StructureFactor(z_2_data)

## ``compute_sf_via_scattering_inetensity``:

### For Poisson point process: 
we know that the pair correlation function and the structure factor of the Poisson point process are equal to 1 so we always plot the line  $𝑦=1$ as a theoretical reference to the Poisson point process 

In [ ]:
wave_length_poisson, scattering_intensity_poisson = sf_poisson.compute_sf_via_scattering_intensity(max_k=15, meshgrid_size=None)

### ``plot_scattering_intensity``:
The method ``plot_scattering_intensity`` take the output of the method ``compute_sf_via_scattering_intensity`` to plot them.
The optional argument ``binning_parameter``correspnding to the parameters used to average the values of the scattering intensity over bins, and ``exact_sf`` is the sctructure factor function if it's known.

In [ ]:
sf_poisson.plot_scattering_intensity(wave_length_poisson, scattering_intensity_poisson, bins=50)

### For Ginibre point process:
as the Ginibre point process lies in a ball window we will use ``restrict_to_cubic_window`` to restricting it into a cubic window.

In [ ]:
plt.plot(ginibre_data.points[:,0], ginibre_data.points[:,1], 'b,')
plt.show()

In [ ]:
R_ginibre = ginibre_data.window.radius
L_ginibre = R_ginibre/np.sqrt(2)
bounds = np.array([[-L_ginibre/2, -L_ginibre/2], [L_ginibre/2, L_ginibre/2]])
ginibre_data_restricted = ginibre_data.restrict_to_window(bounds)

sf_ginibre_2 = StructureFactor(ginibre_data_restricted)

plt.plot(ginibre_data_restricted.points[:,0], ginibre_data_restricted.points[:,1], 'b,')
plt.show()

In [ ]:
wave_length_ginibre, scattering_intensity_ginibre = sf_ginibre_2.compute_sf_via_scattering_intensity(max_k =10, meshgrid_size=300)

In [ ]:
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) # exact structure factor for the Ginibre point process

sf_ginibre_2.plot_scattering_intensity(wave_length_ginibre, scattering_intensity_ginibre, plot_type="all", exact_sf=exact_sf_ginibre, bins=70 )

### For the processus of  Michael Andreas Klatt, Günter Last, D. Yogeshwaran that we will denoted by kly defined in https://arxiv.org/abs/1810.00265

In [ ]:
wave_length_kly, si_kly = sf_kly.compute_sf_via_scattering_intensity(max_k=20)

In [ ]:
sf_kly.plot_scattering_intensity(wave_length_kly, si_kly, plot_type="plot", bins=50 )

In [ ]:
wave_length_kly, si_kly = sf_kly.compute_sf_via_scattering_intensity(max_k=10, meshgrid_size=100)

In [ ]:
sf_kly.plot_scattering_intensity(wave_length_kly, si_kly, plot_type="all", bins=40)

### For a lattice $\mathbb{Z}^2$

In [ ]:
wave_length_z_2, si_z_2 = sf_z_2.compute_sf_via_scattering_intensity(max_k=30)

In [ ]:
sf_z_2.plot_scattering_intensity(wave_length_z_2, si_z_2, plot_type="plot", bins=50 )

In [ ]:
wave_length_z2, si_z2 = sf_z_2.compute_sf_via_scattering_intensity(max_k=100, meshgrid_size=100)

In [ ]:
sf_z_2.plot_scattering_intensity(wave_length_z2, si_z2, plot_type="all", bins=40)

## ``compute_structure_factor_via_hankel``: 
Before using the method ``compute_structure_factor_via_hankel`` you have to approximate the pair correlation function (if we don't know them). 
For this purpose we will use the method ``compute_pcf``.


### ``compute_pcf``:
This method approximate the pair correlation function using the 2 methods ``pcf.ppp`` and ``pcf.fv`` of the R packadge, we will give examples using the 2 methods

### for a Poisson point process

In [ ]:
pcf_ppp_poisson = sf_poisson.compute_pcf(method="ppp",  correction="iso")
pcf_ppp_poisson

In [ ]:
sf_poisson.plot_pcf(pcf_ppp_poisson, figsize=(8,5))

### `` interpolate_pcf``:
Or ``compute_sf_via_scattering_intensity`` take the pair correlationf function as input and not a discret evaluation, we should interpolate the pcf discret sample. This can be done using ``interpolate_pcf``.

In [ ]:
domain, poisson_pcf_func = sf_poisson.interpolate_pcf(r=pcf_ppp_poisson["r"], pcf_r=pcf_ppp_poisson["iso"], clean=True) 
domain

### compute_sf_via_hankel:
This method approximate the symmetric fourier transform presente in the definition of the structure factor using the 2 methods: the first is  ``Ogata`` which is based on the quadrature dirived by Ogata in https://joss.theoj.org/papers/10.21105/joss.01397 to approximate integral with Bessel function and the second is ``BaddourChouinard`` which is an approximation of the hakel transform by quasi-hankel transfrm descibed in https://openresearchsoftware.metajnl.com/articles/10.5334/jors.82/, we will give examples using the 2 methods.

In [ ]:
k = np.linspace(2,15, 100)
rmax = domain["r_max"]
k, sf_poisson_Ogata = sf_poisson.compute_sf_via_hankel(poisson_pcf_func, k=k, method="Ogata",
                                                       step_size=0.1, nb_points=1000, r_max=rmax)
sf_poisson.k_min

In [ ]:
sf_poisson.plot_sf_via_hankel(k, sf_poisson_Ogata)

In [ ]:
k_, sf_poisson_baddour = sf_poisson.compute_sf_via_hankel(poisson_pcf_func, 
                                                                       method ="BaddourChouinard",
                                                                       r_max=rmax, nb_points=600)


In [ ]:
L_poisson = 200 #length side of the cube containing the Poisson point process
bounds = np.array([[-L_poisson/2, -L_poisson/2], [L_poisson/2, L_poisson/2]]) #bounds of the box window containing the point process
window = BoxWindow(bounds) #box window containing the point process

intensity_poisson = 1 #intensity of the Poisson point process
process = HomogeneousPoissonPointProcess(intensity=intensity_poisson) 
points_3 = process.generate_sample(window=window) #Poisson point process in the cubic window

poisson_data_3 = PointPattern(points_3, window, intensity=1)

sf_poisson_3 = StructureFactor(poisson_data_3)

In [ ]:
pcf_fv_poisson = sf_poisson_3.compute_pcf(method="fv", Kest=dict(rmax=60), fv=dict( spar=0.1,  method="c"))
pcf_fv_poisson

In [ ]:
sf_poisson_3.plot_pcf(pcf_fv_poisson, figsize=(8,5))

In [ ]:
domain, poisson_fv_func = sf_poisson.interpolate_pcf(r=pcf_fv_poisson["r"], pcf_r=pcf_fv_poisson["pcf"], clean=True) 
rmax = domain["r_max"]

k_3, sf_poisson_3_baddour = sf_poisson_3.compute_sf_via_hankel(poisson_fv_func, 
                                                                       method ="BaddourChouinard",
                                                                       r_max=rmax, nb_points=600)
sf_poisson_3.plot_sf_via_hankel(k_, sf_poisson_3_baddour)

In [ ]:
k = np.linspace(1,15, 100)
rmax = domain["r_max"]
k_3, sf_poisson_3_Ogata = sf_poisson_3.compute_sf_via_hankel(poisson_fv_func, k=k, method="Ogata",
                                                       step_size=0.1, nb_points=1000, r_max=rmax)
k_min = sf_poisson_3.k_min
print(k_min)

In [ ]:
sf_poisson_3.plot_sf_via_hankel(k=k, sf=sf_poisson_3_Ogata, k_min=k_min)

### For Ginibre

In [ ]:
pcf_ppp_ginibre = sf_ginibre.compute_pcf(method="ppp", correction="all")
pcf_ppp_ginibre

In [ ]:
exact_pcf_ginibre = lambda x : 1 - np.exp(-x**2)

sf_ginibre.plot_pcf(pcf_ppp_ginibre, exact_pcf=exact_pcf_ginibre, figsize=(8,5))

In [ ]:
domain_1, ginibre_pcf_ppp_func = sf_ginibre.interpolate_pcf(r=pcf_ppp_ginibre["r"], pcf_r=pcf_ppp_ginibre["iso"], clean=True) 
rmax_ginibre = domain_1["r_max"]

k_ginibre, sf_ginibre_baddour = sf_ginibre.compute_sf_via_hankel(ginibre_pcf_ppp_func, 
                                                                       method ="BaddourChouinard",
                                                                       r_max=rmax_ginibre, nb_points=800)
sf_ginibre.plot_sf_via_hankel(k_ginibre, sf_ginibre_baddour, exact_sf=exact_sf_ginibre)

In [ ]:
k_ginibre2, sf_ginibre_baddour2 = sf_ginibre.compute_sf_via_hankel(exact_pcf_ginibre, 
                                                                       method ="BaddourChouinard",
                                                                       r_max=200, nb_points=800)
sf_ginibre.plot_sf_via_hankel(k_ginibre2, sf_ginibre_baddour2, exact_sf=exact_sf_ginibre)

In [ ]:
pcf_fv_ginibre = sf_ginibre.compute_pcf(method="fv", Kest=dict(rmax=30), fv=dict( spar=0.1,  method="b"))
pcf_fv_ginibre

In [ ]:
sf_ginibre.plot_pcf(pcf_fv_ginibre, exact_pcf=exact_pcf_ginibre, figsize=(8,5))

In [ ]:
domain_3, ginibre_pcf_fv_func3 = sf_ginibre.interpolate_pcf(r=pcf_fv_ginibre["r"], pcf_r=pcf_fv_ginibre["pcf"], clean=True) 
rmax_ginibre3 = domain_3["r_max"]

k = np.linspace(2,10, 1000)

k_ginibre3, sf_ginibre_Ogata3 = sf_ginibre.compute_sf_via_hankel(ginibre_pcf_fv_func3, k=k,
                                                                       step_size=0.1, nb_points=1000, r_max=rmax_ginibre3)
k_min = sf_ginibre.k_min
print(k_min)
sf_ginibre.plot_sf_via_hankel(k_ginibre3, sf_ginibre_Ogata3, k_min=k_min,  exact_sf=exact_sf_ginibre,)

In [ ]:
k = np.linspace(0.5,10, 1000)
k_ginibre4, sf_ginibre_Ogata4 = sf_ginibre.compute_sf_via_hankel(exact_pcf_ginibre, k=k,
                                                                       step_size=0.01, nb_points=1000)
sf_ginibre.plot_sf_via_hankel(k_ginibre4, sf_ginibre_Ogata4,exact_sf=exact_sf_ginibre,)

### For kly

In [ ]:
pcf_ppp_kly = sf_kly.compute_pcf(method="ppp", stoyan=0.2, correction="good")
pcf_ppp_kly

In [ ]:
sf_kly.plot_pcf(pcf_ppp_kly, figsize=(8,5))

In [ ]:
domain, kly_pcf_ppp_func = sf_kly.interpolate_pcf(r=pcf_ppp_kly["r"], pcf_r=pcf_ppp_kly["trans"], clean=True) 
rmax_kly = domain["r_max"]

k = np.linspace(2,10, 1000)

k_kly, sf_kly_1 = sf_kly.compute_sf_via_hankel(kly_pcf_ppp_func, k=k, method="Ogata",
                                                step_size=0.1, nb_points=1000, r_max=rmax_kly)
k_min = sf_kly.k_min
print(k_min)
sf_kly.plot_sf_via_hankel(k_kly, sf_kly_1, k_min=k_min, )

In [ ]:
k_kly, sf_kly_1 = sf_kly.compute_sf_via_hankel(kly_pcf_ppp_func, method ="BaddourChouinard",
                                                r_max=rmax_kly, nb_points=800)

sf_kly.plot_sf_via_hankel(k_kly, sf_kly_1 )

In [ ]:
pcf_fv_kly = sf_kly.compute_pcf(method="fv", Kest=dict(rmax=50), fv=dict( spar=0.3,  method="b"))
pcf_fv_kly

In [ ]:
domain, kly_pcf_fv_func = sf_kly.interpolate_pcf(r=pcf_fv_kly["r"], pcf_r=pcf_fv_kly["pcf"], clean=True) 
rmax_kly = domain["r_max"]

k_kly_2, sf_kly_2 = sf_kly.compute_sf_via_hankel(kly_pcf_fv_func, method ="BaddourChouinard",
                                                r_max=rmax_kly, nb_points=800)
sf_kly.plot_sf_via_hankel(k_kly_2, sf_kly_2 )


In [ ]:
k = np.linspace(2,30, 1000)
k_kly_3, sf_kly_3 = sf_kly.compute_sf_via_hankel(kly_pcf_fv_func, k=k, method="Ogata",
                                                step_size=0.07, nb_points=1000, r_max=rmax_kly)
k_min = sf_kly.k_min
print(k_min)
sf_kly.plot_sf_via_hankel(k_kly_3, sf_kly_3, k_min )